In [1]:
import pandas as pd
import numpy as np
import os
from os.path import join as pjoin
import labelHelper as lh
from sklearn.model_selection import train_test_split
import random

Import an awesome Labelhelper


In [12]:
def get_raw_feature_by_folder_and_videoNumbers(video_data_folder,video_number):
    feature=[]
    print("load feature from files, there are %d videos"%len(video_number))
    for number in video_number:
        file_name = "raw_{}.npy".format(str(number))
        each_video= pjoin(video_data_folder,file_name)
        try:
            feature.append(np.load(each_video))
        except:
            print("Video %d did not find" %number)
            video_number = video_number[video_number!=number]
    
    print("after deop: ",len(video_number))
    return feature,video_number

def get_diff_and_hstack_to_orginal_data(X,time_lag=2,move_threshold=200):
    original=X[:-time_lag]
    modified=X[time_lag:]
    result=modified-original
    ## threshold
    # consider there was no top in the first image and it showed up in the next image
    # the difference would be huge, which was not ideal.
    # so we need to filter these extrem value 
    result[np.abs(result)>move_threshold]=0
    
    return np.hstack((X[time_lag:],result))

##from each video sample video clips with size=window_L. you can specify stride 
def make_video_clips(matrix,window_L,stride):
    alldata=[]
    total_frame=matrix.shape[0]
    index=[n for n in range(1,total_frame,stride)]
    for start_index in index:
        if start_index+window_L> total_frame:
            break
#         print(start_index)
        each_clip_data=matrix[start_index:start_index+window_L]
#         each_clip_data=np.transpose(each_clip_data)
#         print(each_clip_data.shape)
        alldata.append(each_clip_data)
    return np.array(alldata)

def make_video_feature_list(raw_feature,video_clips_length,time_lag,move_threshold,stride):
    training_data=[]
    for each_video in raw_feature:
        new_feature=get_diff_and_hstack_to_orginal_data(each_video,time_lag,move_threshold)
        video_clip=make_video_clips(new_feature,video_clips_length,stride)
#         print(video_clip.shape)
        training_data.append((video_clip))
    return np.array(training_data)


def stack_videoClips_and_getLabelByVideoNumber(training_data_list,video_number,label):
    if len(training_data_list)!= len(video_number):
        print('the length of training_data_list is not equal to video_number_list!')
        return 
    final_data=training_data_list[0]
    final_label=[label.get_video_mean_label(video_number[0]) for _ in range(final_data.shape[0])]

    for i in range(1,len(training_data_list)):
        final_data=np.vstack((final_data,training_data_list[i]))
        for j in range(training_data_list[i].shape[0]):
            final_label.append(label.get_video_mean_label(video_number[i]))
    return final_data,np.array(final_label)


def train_test_split_by_videoNumber(total_data,video_number,test_ratio,label):
    index=np.array(range(0,len(video_number)))    
    train_video_index,test_video_index= train_test_split(index,test_size=test_ratio)
    
    train_video=total_data[train_video_index]
    test_video=total_data[test_video_index]
    
    train_video_number=video_number[train_video_index]
    test_video_number=video_number[test_video_index]
    
    
    train_video,train_video_label=stack_videoClips_and_getLabelByVideoNumber(train_video,train_video_number,label)
    test_video,test_video_label=stack_videoClips_and_getLabelByVideoNumber(test_video,test_video_number,label)
    return train_video,train_video_label,test_video,test_video_label


In [3]:
ll=lh.Label('..//2019_fall_labels.csv')
video_number = ll.get_video_number()
print(video_number)
print("before deop: ",len(video_number))

[ 11  16  49  59  74  78  91 113 130 143 152 164 171 176 192 194 207 222
 225 226 236 237 240 253 257 258 267 276 277 283 294 296 301 302 333 344
 345 358 361 368 378 384 401 414 417 425 427 436 440 441 455 457 460 471
 486 498 503 507 527 530 536 537 538 539 557 559 562 578 599 615 632 689
 716 717 742 761 820 861 881 895 921 924 928 942]
before deop:  84


In [7]:
folder = "C:\\2019_fall_video_features"

raw_feature,video_number=get_raw_feature_by_folder_and_videoNumbers(folder,video_number)
print("after deop: ",len(video_number))
len(raw_feature)

Video 924 did not find
after deop:  83


83

In [8]:
# ####################################### set video clips parameters
# ############################################### make new 28-d feature
video_clips_length=30
time_lag=2
move_threshold=150
stride=video_clips_length

training_data_list=prepare_training_data_list(raw_feature,video_clips_length,time_lag,move_threshold,stride)

In [6]:
# # x,y = stack_videoClips_and_getLabelByVideoNumber(training_data_list,video_number,ll)
# train_x,train_y,test_x,test_y=train_test_split_by_videoNumber(training_data_list,video_number,0.2,ll)

In [16]:
training_data_list[[0,1,2]].shape

(3,)